<a href="https://colab.research.google.com/github/shamsakhoja7-max/msba-portfolio-shamsakhoja/blob/main/Final_K_5_Segments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports
import os
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_excel('/content/drive/MyDrive/Class - PA/num_churn_df_data_2.xlsx')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df["left_flag"] = (df["left_flag"] == "Yes").astype(int)


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import pandas as pd
import numpy as np

In [ ]:


# Remove redundant/dummy variables, keep core features
core_features = [
    # Customer tenure & value
    'tenure_mo',
    'monthly_fee',
    'total_billed',

    # Usage patterns
    'avg_gb_download',
    'num_add_ons',
    'long_dist_fees_total',

    'extra_data_fees_total',

    # Key problem indicators
    'unexpected_cost_burden',
    'overage_ratio',

    # Service characteristics
    'has_streaming',
    'unlimited_data_opt',
    'e_bill_opt_in',

    # Demographics
    'age_years',
    'has_dependents',

    # Instead of ALL payment method dummies, create composite:
    # (manually calculated in next step)
]

# Create composite features instead of using all dummies
df['auto_payment'] = (df['pay_method_Bank transfer (automatic)'] +
                        df['pay_method_Credit card (automatic)'])

df['fiber_user'] = df['internet_tech_Fiber Optic']

df['month_to_month'] = df['contract_term_Month-to-month']

# Final feature set (15-20 features)
selected_features = [
    'tenure_mo', 'monthly_fee', 'avg_gb_download', 'num_add_ons',
    'long_dist_fees_total', 'unexpected_cost_burden',
    'has_streaming', 'unlimited_data_opt', 'e_bill_opt_in',
    'age_years', 'auto_payment', 'fiber_user', 'month_to_month',
    'has_add_on', 'has_dependents'
]

print(f"Using {len(selected_features)} selected features")

# Re-run clustering with selected features
scaler2 = StandardScaler()
X_scaled2 = scaler2.fit_transform(df[selected_features])

# Try K=3 first (since K=2 performed better)
for k in [2, 3, 4, 5]:
    kmeans_test = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels_test = kmeans_test.fit_predict(X_scaled2)
    sil = silhouette_score(X_scaled2, labels_test)
    db = davies_bouldin_score(X_scaled2, labels_test)
    print(f"K={k}: Silhouette={sil:.3f}, Davies-Bouldin={db:.3f}")

Using 15 selected features
K=2: Silhouette=0.233, Davies-Bouldin=1.566
K=3: Silhouette=0.204, Davies-Bouldin=1.775
K=4: Silhouette=0.178, Davies-Bouldin=1.952
K=5: Silhouette=0.166, Davies-Bouldin=1.910


In [ ]:

# RUN K=3 CLUSTERING


kmeans_k3 = KMeans(n_clusters=3, random_state=42, n_init=10)
df['segment_k3'] = kmeans_k3.fit_predict(X_scaled2)

print("=" * 60)
print("K=3 SEGMENT PROFILES")
print("=" * 60)

# Profile the 3 segments
profile_features = [
 'tenure_mo', 'monthly_fee', 'unexpected_cost_burden',
    'avg_gb_download', 'long_dist_fees_total', 'num_add_ons',
    'unlimited_data_opt', 'auto_payment', 'fiber_user', 'month_to_month',
    'e_bill_opt_in', 'has_streaming', 'age_years'
]

segment_profiles_k3 = df.groupby('segment_k3')[profile_features].mean().round(2)
print("\nMean values by segment:")
print(segment_profiles_k3)

# Segment sizes
print("\n" + "=" * 60)
print("SEGMENT SIZES (K=3)")
print("=" * 60)
segment_counts_k3 = df['segment_k3'].value_counts().sort_index()
segment_pct_k3 = (segment_counts_k3 / len(df) * 100).round(1)

for seg in range(3):
    print(f"Segment {seg}: {segment_counts_k3[seg]:,} customers ({segment_pct_k3[seg]}%)")

print("\n" + "=" * 60)
print("COMPARING K=2 vs K=3")
print("=" * 60)

# Also run K=2 for comparison
kmeans_k2 = KMeans(n_clusters=2, random_state=42, n_init=10)
df['segment_k2'] = kmeans_k2.fit_predict(X_scaled2)

segment_profiles_k2 = df.groupby('segment_k2')[profile_features].mean().round(2)
print("\nK=2 Segment Profiles:")
print(segment_profiles_k2)

segment_counts_k2 = df['segment_k2'].value_counts().sort_index()
segment_pct_k2 = (segment_counts_k2 / len(df) * 100).round(1)

print("\nK=2 Segment Sizes:")
for seg in range(2):
    print(f"Segment {seg}: {segment_counts_k2[seg]:,} customers ({segment_pct_k2[seg]}%)")

K=3 SEGMENT PROFILES

Mean values by segment:
            tenure_mo  monthly_fee  unexpected_cost_burden  avg_gb_download  \
segment_k3                                                                    
0               15.91        70.74                    0.30            24.33   
1               54.73        85.27                    0.27            28.26   
2               29.97        22.30                    1.20             0.98   

            long_dist_fees_total  num_add_ons  unlimited_data_opt  \
segment_k3                                                          
0                         308.62         0.92                0.88   
1                        1267.40         2.55                0.86   
2                         766.52         0.02                0.01   

            auto_payment  fiber_user  month_to_month  e_bill_opt_in  \
segment_k3                                                            
0                   0.26        0.59            0.96           0.71   

In [ ]:

# RUN K=4 CLUSTERING


kmeans_k4 = KMeans(n_clusters=4, random_state=42, n_init=10)
df['segment'] = kmeans_k4.fit_predict(X_scaled2)

print("=" * 60)
print("K=4 SEGMENT PROFILES")
print("=" * 60)

# Profile the 4 segments
profile_features = [
    'tenure_mo', 'monthly_fee', 'unexpected_cost_burden',
    'avg_gb_download', 'long_dist_fees_total', 'num_add_ons',
    'unlimited_data_opt', 'auto_payment', 'fiber_user', 'month_to_month',
    'e_bill_opt_in', 'has_streaming', 'age_years'
]

segment_profiles = df.groupby('segment')[profile_features].mean().round(2)
print("\nMean values by segment:")
print(segment_profiles)

# Segment sizes
print("\n" + "=" * 60)
print("SEGMENT SIZES (K=4)")
print("=" * 60)
segment_counts = df['segment'].value_counts().sort_index()
segment_pct = (segment_counts / len(df) * 100).round(1)

for seg in range(4):
    print(f"Segment {seg}: {segment_counts[seg]:,} customers ({segment_pct[seg]}%)")

print(f"\nTotal: {len(df):,} customers")

# Detailed profiling with percentages
print("\n" + "=" * 60)
print("DETAILED SEGMENT CHARACTERISTICS")
print("=" * 60)

detailed_profile = df.groupby('segment').agg({
    'tenure_mo': 'mean',
    'monthly_fee': 'mean',
    'unexpected_cost_burden': 'mean',
    'avg_gb_download': 'mean',
    'long_dist_fees_total': 'mean',
    'num_add_ons': 'mean',
    'unlimited_data_opt': 'mean',  # Proportion
    'auto_payment': 'mean',         # Proportion
    'fiber_user': 'mean',           # Proportion
    'month_to_month': 'mean',       # Proportion
    'age_years': 'mean'
}).round(2)

print(detailed_profile)

# Convert proportions to percentages for clarity
print("\n" + "=" * 60)
print("KEY PERCENTAGES BY SEGMENT")
print("=" * 60)

pct_profile = df.groupby('segment').agg({
    'unlimited_data_opt': lambda x: f"{x.mean()*100:.0f}%",
    'auto_payment': lambda x: f"{x.mean()*100:.0f}%",
    'fiber_user': lambda x: f"{x.mean()*100:.0f}%",
    'month_to_month': lambda x: f"{x.mean()*100:.0f}%",
    'has_streaming': lambda x: f"{x.mean()*100:.0f}%",
    'e_bill_opt_in': lambda x: f"{x.mean()*100:.0f}%"
})

pct_profile.columns = ['Unlimited %', 'Autopay %', 'Fiber %', 'Month-to-Month %', 'Streaming %', 'E-Bill %']
print(pct_profile)

K=4 SEGMENT PROFILES

Mean values by segment:
         tenure_mo  monthly_fee  unexpected_cost_burden  avg_gb_download  \
segment                                                                    
0            30.37        21.82                    1.21             0.53   
1            17.95        50.79                    0.29            27.71   
2            57.59        88.09                    0.28            27.98   
3            18.21        85.68                    0.31            22.64   

         long_dist_fees_total  num_add_ons  unlimited_data_opt  auto_payment  \
segment                                                                        
0                      778.05         0.01                0.00          0.43   
1                      210.74         1.29                0.88          0.31   
2                     1377.86         2.63                0.86          0.70   
3                      440.13         0.83                0.86          0.25   

         fiber_u

In [ ]:
# RUN K=4 CLUSTERING


kmeans_k4 = KMeans(n_clusters=4, random_state=42, n_init=20)
df['segment'] = kmeans_k4.fit_predict(X_scaled2)

print("=" * 60)
print("K=4 SEGMENT PROFILES")
print("=" * 60)

# Profile the 4 segments
profile_features = [
    'tenure_mo', 'monthly_fee', 'unexpected_cost_burden',
    'avg_gb_download', 'long_dist_fees_total', 'num_add_ons',
    'unlimited_data_opt', 'auto_payment', 'fiber_user', 'month_to_month',
    'e_bill_opt_in', 'has_streaming', 'age_years'
]

segment_profiles = df.groupby('segment')[profile_features].mean().round(2)
print("\nMean values by segment:")
print(segment_profiles)

# Segment sizes
print("\n" + "=" * 60)
print("SEGMENT SIZES (K=4)")
print("=" * 60)
segment_counts = df['segment'].value_counts().sort_index()
segment_pct = (segment_counts / len(df) * 100).round(1)

for seg in range(4):
    print(f"Segment {seg}: {segment_counts[seg]:,} customers ({segment_pct[seg]}%)")

print(f"\nTotal: {len(df):,} customers")

# Detailed profiling with percentages
print("\n" + "=" * 60)
print("DETAILED SEGMENT CHARACTERISTICS")
print("=" * 60)

detailed_profile = df.groupby('segment').agg({
    'tenure_mo': 'mean',
    'monthly_fee': 'mean',
    'unexpected_cost_burden': 'mean',
    'avg_gb_download': 'mean',
    'long_dist_fees_total': 'mean',
    'num_add_ons': 'mean',
    'unlimited_data_opt': 'mean',  # Proportion
    'auto_payment': 'mean',         # Proportion
    'fiber_user': 'mean',           # Proportion
    'month_to_month': 'mean',       # Proportion
    'age_years': 'mean'
}).round(2)

print(detailed_profile)

# Convert proportions to percentages for clarity
print("\n" + "=" * 60)
print("KEY PERCENTAGES BY SEGMENT")
print("=" * 60)

pct_profile = df.groupby('segment').agg({
    'unlimited_data_opt': lambda x: f"{x.mean()*100:.0f}%",
    'auto_payment': lambda x: f"{x.mean()*100:.0f}%",
    'fiber_user': lambda x: f"{x.mean()*100:.0f}%",
    'month_to_month': lambda x: f"{x.mean()*100:.0f}%",
    'has_streaming': lambda x: f"{x.mean()*100:.0f}%",
    'e_bill_opt_in': lambda x: f"{x.mean()*100:.0f}%"
})

pct_profile.columns = ['Unlimited %', 'Autopay %', 'Fiber %', 'Month-to-Month %', 'Streaming %', 'E-Bill %']
print(pct_profile)

print(f"K={k}: Silhouette={sil:.3f}, Davies-Bouldin={db:.3f}")

K=4 SEGMENT PROFILES

Mean values by segment:
         tenure_mo  monthly_fee  unexpected_cost_burden  avg_gb_download  \
segment                                                                    
0            57.61        88.16                    0.28            27.99   
1            18.21        85.68                    0.31            22.64   
2            18.04        50.82                    0.29            27.68   
3            30.37        21.82                    1.21             0.53   

         long_dist_fees_total  num_add_ons  unlimited_data_opt  auto_payment  \
segment                                                                        
0                     1379.87         2.63                0.86          0.70   
1                      440.13         0.83                0.86          0.25   
2                      211.84         1.29                0.88          0.31   
3                      778.05         0.01                0.00          0.43   

         fiber_u

In [ ]:
# RUN K=4 CLUSTERING


kmeans_k4 = KMeans(n_clusters=4, random_state=42, n_init=20)
df['segment'] = kmeans_k4.fit_predict(X_scaled2)

print("=" * 60)
print("K=4 SEGMENT PROFILES")
print("=" * 60)

# Profile the 4 segments
profile_features = [
    'tenure_mo', 'monthly_fee', 'unexpected_cost_burden',
    'avg_gb_download', 'long_dist_fees_total', 'num_add_ons',
    'unlimited_data_opt', 'auto_payment', 'fiber_user', 'month_to_month',
    'e_bill_opt_in', 'has_streaming', 'age_years'
]

segment_profiles = df.groupby('segment')[profile_features].mean().round(2)
print("\nMean values by segment:")
print(segment_profiles)

# Segment sizes
print("\n" + "=" * 60)
print("SEGMENT SIZES (K=4)")
print("=" * 60)
segment_counts = df['segment'].value_counts().sort_index()
segment_pct = (segment_counts / len(df) * 100).round(1)

for seg in range(4):
    print(f"Segment {seg}: {segment_counts[seg]:,} customers ({segment_pct[seg]}%)")

print(f"\nTotal: {len(df):,} customers")

# Detailed profiling with percentages
print("\n" + "=" * 60)
print("DETAILED SEGMENT CHARACTERISTICS")
print("=" * 60)

detailed_profile = df.groupby('segment').agg({
    'tenure_mo': 'mean',
    'monthly_fee': 'mean',
    'unexpected_cost_burden': 'mean',
    'avg_gb_download': 'mean',
    'long_dist_fees_total': 'mean',
    'num_add_ons': 'mean',
    'unlimited_data_opt': 'mean',  # Proportion
    'auto_payment': 'mean',         # Proportion
    'fiber_user': 'mean',           # Proportion
    'month_to_month': 'mean',       # Proportion
    'age_years': 'mean'
}).round(2)

print(detailed_profile)

# Convert proportions to percentages for clarity
print("\n" + "=" * 60)
print("KEY PERCENTAGES BY SEGMENT")
print("=" * 60)

pct_profile = df.groupby('segment').agg({
    'unlimited_data_opt': lambda x: f"{x.mean()*100:.0f}%",
    'auto_payment': lambda x: f"{x.mean()*100:.0f}%",
    'fiber_user': lambda x: f"{x.mean()*100:.0f}%",
    'month_to_month': lambda x: f"{x.mean()*100:.0f}%",
    'has_streaming': lambda x: f"{x.mean()*100:.0f}%",
    'e_bill_opt_in': lambda x: f"{x.mean()*100:.0f}%"
})

pct_profile.columns = ['Unlimited %', 'Autopay %', 'Fiber %', 'Month-to-Month %', 'Streaming %', 'E-Bill %']
print(pct_profile)

print(f"K={k}: Silhouette={sil:.3f}, Davies-Bouldin={db:.3f}")

K=4 Advantages:
✅ All 4 segments tell clear, actionable stories
✅ Better size distribution (22%, 21%, 30%, 27% - more balanced)
✅ Each segment has distinct intervention needs
K=4 Final Segment Profiles:
Segment 0: "Legacy High-Risk" (22.1%) 🚨

Tenure: 30 months, Fee: $22, Cost burden: 1.21 (HIGHEST!)
0% Unlimited, 0% Fiber, 0% Streaming, 0.53 GB data
35% month-to-month, 43% autopay
Story: Older customers stuck on cheapest legacy DSL/Cable plans with severe overage problems despite minimal usage
Action: Force migrate to unlimited or better plans

Segment 1: "At-Risk New Customers" (21.4%) ⚠️

Tenure: 18 months (SHORT), Fee: $51, Cost burden: 0.29
88% Unlimited, only 1% Fiber, 37% Streaming
79% month-to-month (HIGHEST!), only 31% autopay
Story: New customers who adopted unlimited to stop overages, but on slow infrastructure with no commitment
Action: Fiber upgrades, autopay incentives, contract offers

Segment 2: "Premium Engaged" (29.5%) ✅

Tenure: 58 months (LONGEST), Fee: $88, Cost burden: 0.28 (LOW)
86% Unlimited, 56% Fiber, 86% Streaming
Only 12% month-to-month, 70% autopay
Story: Long-term, high-value customers on good infrastructure with full bundles and commitment
Action: Protect, upsell premium services

Segment 3: "Premium Fiber Loyalists" (27.0%) ✅

Tenure: 18 months, Fee: $86, Cost burden: 0.31 (LOW)
86% Unlimited, 99% Fiber (HIGHEST!), 60% Streaming
98% month-to-month but only 25% autopay
Story: Premium Fiber customers, likely newer but locked in by infrastructure quality despite month-to-month
Action: Convert to contracts with incentives, push autopay